# Regression hw in week 14

**info**

SuJinhua 2017201620

**Dataset & DIY function**

In [2]:
import pandas as pd
import numpy as np
df = pd.read_excel('table10-13.xlsx')
print(df.head())
df.describe()

   num  x1  x2  x3   x4  y
0    1  95  65   1  600  2
1    2  63  62   0  850  1
2    3  82  53   0  700  2
3    4  60  88   0  850  3
4    5  72  65   1  750  1


,num,x1,x2,x3,x4,y
count,40.000000,40.000000,40.000000,40.000000,40.00000,40.000000
mean,20.500000,77.875000,77.675000,0.450000,823.75000,1.750000
std,11.690452,10.811171,12.081746,0.503831,206.30681,0.742484
min,1.000000,60.000000,53.000000,0.000000,500.00000,1.000000
25%,10.750000,71.750000,65.000000,0.000000,687.50000,1.000000
50%,20.500000,79.000000,77.000000,0.000000,800.00000,2.000000
75%,30.250000,86.000000,88.000000,1.000000,950.00000,2.000000
max,40.000000,96.000000,96.000000,1.000000,1500.00000,3.000000


In [3]:
def linear_regression(x,y):
    x = np.hstack((np.ones((x.shape[0],1)),x))
    beta = np.linalg.inv(x.T.dot(x)).dot(x.T.dot(y))
    return beta
def logit_regression(x,y,t=100,init_beta =False,beta = None,method = 'newton',lr = 0.001):
    x = np.hstack((np.ones((x.shape[0],1)),x))
    if init_beta == False:
        beta = np.zeros((x.shape[1],1))
    def p1(x,y,beta):
        return 1/(1+np.exp(-1 * x.dot(beta)))
    def derivate_1(x,y,beta):
        return -1 * x.T.dot((y-p1(x,y,beta).T).T)
    def derivate_2(x,y,beta):
        tmp = p1(x,y,beta)
        tmp = tmp*(1-tmp)
        return x.T.dot(x*tmp)
    if method == 'newton':
        for i in range(t):
            beta = beta - np.linalg.inv(derivate_2(x,y,beta)).dot(derivate_1(x,y,beta))
    elif method == 'simple':
        for i in range(t):
            beta = beta - lr * derivate_1(x,y,beta)
    return beta

In [6]:
beta_l = []
for i in range(3-1):
    df['y=%s'%(i+2)] = (df['y'] == i+2).astype(int)
    df_tmp = df[df['y'].apply(lambda x:True if x in [1,i+2] else False)]
    X = df_tmp[['x'+str(i+1) for i in range(4)]].to_numpy()
    y = df_tmp['y=%s'%(i+2)].to_numpy()
    beta_l.append(logit_regression(X,y))

In [53]:
Beta = np.hstack(beta_l)

In [34]:
def p1(x,beta):
    x = np.hstack([np.ones((len(x),1)),x])
    return np.exp(x.dot(beta))
p = p1(df[['x%s'%(i+1) for i in range(4)]].to_numpy(),Beta)
p = np.hstack([np.ones((p.shape[0],1)),p])

In [35]:
p = (p.T / p.sum(axis = 1)).T

In [60]:
my_pred =(np.argsort(p)+1)[:,2]
my_pred

array([2, 1, 1, 3, 1, 3, 3, 2, 1, 2, 2, 2, 2, 1, 1, 3, 1, 2, 3, 2, 2, 1,
       2, 2, 2, 2, 1, 1, 1, 2, 1, 2, 1, 3, 1, 2, 1, 1, 3, 2])

In [61]:
real = df['y'].to_numpy()
real

array([2, 1, 2, 3, 1, 3, 2, 2, 1, 1, 1, 2, 3, 2, 3, 2, 1, 2, 2, 3, 2, 1,
       2, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 3, 1, 2, 1, 1, 3, 2])

In [68]:
matrix = np.zeros((3,3))
for i in range(len(real)):
    matrix[real[i]-1][my_pred[i]-1] += 1
pd.DataFrame(matrix.astype(int),index=['real_%s'%(i+1) for i in range(3)],columns=['pred_%s'%(i +1) for i in range(3)])

,pred_1,pred_2,pred_3
real_1,12,5,0
real_2,3,10,3
real_3,1,2,4


In [51]:
res = pd.DataFrame(Beta,index = ['const','专业课x1','英语x2','性别x3','月生活费x4'],\
                   columns=['logit 2(base 1)','logit 3(base 1)'])
res

,logit 2(base 1),logit 3(base 1)
const,-20.255322,-24.510588
专业课x1,0.185051,0.006293
英语x2,0.021993,0.184559
性别x3,0.778040,-0.556140
月生活费x4,0.004702,0.010588


**conclusion**

I perform the single logit regression on each target label(2,3) based on the label=1. Then I use softmax function to determine the probability of each class for each row of data. My logit model predict 26 correct label of 40 rows of data so its coefficients have some explainary power for graduates' decision. I use working class(lable=1) as base to show two logit model(1v1) coefficients. For graduate who decide to have a further graduation, the score of professional lessons, English class, and the living expenses may have a positive effect on it, while more girls like to go aboard and more boys choose not to go aboard. For those who finally decide to have aboard education, English Language ability and the expense of living may be a good factor to predict that.

**more analysis**

For more coefficient and test info, I will use package statsmodel to show more results.

In [66]:
import statsmodels.api as sma
for i in range(3-1):
    df_tmp = df[df['y'].apply(lambda x:True if x in [1,i+2] else False)]
    X = df_tmp[['x'+str(i+1) for i in range(4)]].to_numpy()
    y = df_tmp['y=%s'%(i+2)].to_numpy()
    X = np.hstack([np.ones((X.shape[0],1)),X])
    result=sma.Logit(y,X).fit()
    print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.421447
         Iterations 7
                         Results: Logit
Model:              Logit            Pseudo R-squared: 0.392    
Dependent Variable: y                AIC:              37.8155  
Date:               2019-12-19 12:28 BIC:              45.2980  
No. Observations:   33               Log-Likelihood:   -13.908  
Df Model:           4                LL-Null:          -22.859  
Df Residuals:       28               LLR p-value:      0.0012898
Converged:          1.0000           Scale:            1.0000   
No. Iterations:     7.0000                                      
-----------------------------------------------------------------
            Coef.    Std.Err.     z     P>|z|    [0.025    0.975]
-----------------------------------------------------------------
const      -20.2553    7.7839  -2.6022  0.0093  -35.5115  -4.9992
x1           0.1851    0.0709   2.6110  0.0090    0.0461   0.3240
x2      

**more conclusion**

The package give the same result as my DIY function, the significance shows that x1 is the major factor of label=2's decision, while x2 and x4 is the major factor of label=3's decision.